In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from torch.utils.data import DataLoader,TensorDataset
import matplotlib.pyplot as plt
import xlsxwriter
def savexlsx(data,name):
    workbook=xlsxwriter.Workbook('{}.xlsx'.format(name))
    worksheet=workbook.add_worksheet()
    hang = data.shape[0]
    lie = data.shape[1]
    for col in range(int(hang)):
        for row in range(int(lie)):
            worksheet.write(col,row,data[col,row])
    workbook.close()
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch':[], 'epoch':[]}
        self.accuracy = {'batch':[], 'epoch':[]}
        self.val_loss = {'batch':[], 'epoch':[]}
        self.val_acc = {'batch':[], 'epoch':[]}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('acc'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_acc'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('acc'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_acc'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')
        plt.legend(loc="upper right")
        plt.show()

# 训练

In [ ]:
dataset = pd.read_excel('光谱+面积.xlsx', header = None)

In [ ]:
orignal_data = np.array(dataset)

In [ ]:
orignal_data.shape

In [ ]:
data = orignal_data[:,:473]

In [ ]:
label1 = orignal_data[:,473]

In [ ]:
rank = []

In [ ]:
for i in label1:
    if i == 0:
        rank.append(0)
    elif 0 < i <= 25:
        rank.append(1)
    elif 25 < i <= 50:
        rank.append(2)
    elif 50 < i <= 75:
        rank.append(3)
    elif 75 < i <= 100:
        rank.append(4)

In [ ]:
rank_labels = keras.utils.to_categorical(rank)  # 生成标签

In [ ]:
label2 = orignal_data[:,474]

In [ ]:
#train_x, valid_x, train_y, valid_y = train_test_split(data, label2, test_size=0.1)

In [ ]:
#train_x, valid_x, train_y, valid_y = train_test_split(data, label2, test_size=0.1)

In [ ]:
train_x, valid_x, train_y, valid_y = train_test_split(data, rank_labels, test_size=0.1)

t_y = train_y.reshape(-1,1)
v_y = valid_y.reshape(-1,1)

savexlsx(train_x,'train_x')
savexlsx(valid_x,'valid_x')
savexlsx(t_y,'train_y')
savexlsx(v_y,'valid_y')

In [ ]:
savexlsx(train_x,'train_x')
savexlsx(valid_x,'valid_x')
savexlsx(train_y,'train_y')
savexlsx(valid_y,'valid_y')

In [ ]:
from keras import regularizers
from keras.layers import Reshape,Add,Multiply,merge,Input,Dense, Activation,Dense,Dropout,Flatten,BatchNormalization,Conv1D
from keras.layers import MaxPooling1D,AveragePooling1D,LeakyReLU,GlobalAveragePooling1D
from keras.models import Sequential,Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
def Res_block(X,filters):
    
    F1,F2 = filters
 
    X_shortcut = X
 
    X = Conv1D(filters = F1, kernel_size = 1, strides = 1)(X)
    X = Dropout(0.5)(X)
    X = Activation('relu')(X)
    X = Conv1D(filters = F2, kernel_size = 3, strides = 1, padding = 'same')(X)
    
    X_shortcut = Conv1D(F2, kernel_size = 1, strides = 1)(X_shortcut)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
 
    return X

In [ ]:
def ResNet_new(input_shape = (473,), classes = 5):
 
    X_input = Input(input_shape)
    X1 = Dense(473, activation = 'relu', name = 'dense1')(X_input)
    X1 = Dropout(0.5)(X1)
    X1 = Dense(256, activation = 'relu', name = 'dense2')(X1)
    X1 = Dropout(0.5)(X1)
    X1 = Dense(473, name = 'dense3')(X1)
    X = Multiply()([X1, X_input])
    #X = X_input
    X = Reshape((473, 1))(X)
    #X = Reshape((473, 1))(X_input)
    12
    X = Res_block(X,filters=[256,128])
    X = Res_block(X,filters=[64,32])
    X = Res_block(X,filters=[32,16])
 
    #X = GlobalAveragePooling1D(name='global_avg_pool')(X)
    X = AveragePooling1D(3, name = 'avg_pool')(X)
    X = Flatten()(X)
    X = Dense(128, activation = 'relu')(X)
    #X = Dropout(0.5)(X)
    X = Dense(64, activation = 'relu')(X)
    X = Dense(classes)(X)
    model = Model(inputs = X_input, outputs = X, name = 'DINet')
    return model

In [ ]:
model = ResNet_new()

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])  

In [ ]:
history=LossHistory()

In [ ]:
model_names = "model_epoch-val_loss.hdf5"

save_model = ModelCheckpoint(filepath= model_names, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
save_model = ModelCheckpoint(filepath= model_names, monitor='val_accuracy', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

In [ ]:
record_h=model.fit(train_x, train_y, epochs=3000,batch_size=256, validation_data=(valid_x, valid_y), callbacks=[history,save_model])

In [ ]:
plt.plot(record_h.history['loss'])
plt.plot(record_h.history['val_loss'])
plt.plot(record_h.history['accuracy'])
plt.plot(record_h.history['val_accuracy'])
plt.title("model loss and acc")
plt.ylabel("loss and acc") 
plt.xlabel("epoch")
plt.legend(["train","test","train_acc","test_acc"],loc="upper right")
plt.savefig("1.jpg")
plt.show()

In [ ]:
loss = record_h.history['loss']
val_loss = record_h.history['val_loss']
acc = record_h.history['acc']
val_acc = record_h.history['val_acc']

In [ ]:
log_data = np.stack((loss, val_loss, acc,val_acc), axis=1)

In [ ]:
savexlsx(log_data,'log')

# 预测

In [2]:
from keras.models import load_model

In [387]:
model_p = load_model('model_epoch_120-val_loss_27.0025.hdf5')

In [388]:
#train_x1, test_x1, train_y1, test_y1 = train_test_split(data, label2, test_size=0.3)

In [389]:
testset = pd.read_excel('test_x.xlsx', header = None)

In [390]:
testset = np.array(testset)

In [391]:
y3 = model_p.predict(testset)
savexlsx(y3,'test_y_2')

In [ ]:
train_x1, test_x1, train_y1, test_y1 = train_test_split(data, label2, test_size=0.3)

In [ ]:
test_y1 = test_y1.reshape(-1,1)
savexlsx(test_x1,'test_x')
savexlsx(test_y1,'test_y')

In [ ]:
y = model_p.predict(train_x)

In [ ]:
savexlsx(y,'train_y_2')

In [ ]:
y2 = model_p.predict(valid_x)

In [ ]:
savexlsx(y2,'valid_y_2')

In [ ]:
y3 = model_p.predict(test_x1)
savexlsx(y3,'test_y_2')

In [ ]:
model_p.summary()

In [ ]:
layer_name = 'dense3'
intermediate_layer_model = Model(inputs=model_p.input,outputs=model_p.get_layer(layer_name).output)#创建的新模型
output1 = intermediate_layer_model.predict(valid_x)
output2 = intermediate_layer_model.predict(train_x)
output3 = intermediate_layer_model.predict(test_x1)

In [ ]:
savexlsx(output1,'valid_attention_value')
savexlsx(output2,'train_attention_value')
savexlsx(output3,'test_attention_value')

In [ ]:
layer_name = 'multiply_5'
intermediate_layer_model = Model(inputs=model_p.input,outputs=model_p.get_layer(layer_name).output)#创建的新模型
output4 = intermediate_layer_model.predict(valid_x)
output5 = intermediate_layer_model.predict(train_x)
output6 = intermediate_layer_model.predict(test_x1)

In [ ]:
savexlsx(output4,'valid_attention_out')
savexlsx(output5,'train_attention_out')
savexlsx(output6,'test_attention_out')